In [566]:
import cv2
import numpy as np
import os
import glob
from numpy import linalg as la
import math
from scipy.ndimage import filters

Parameters

In [567]:
# focal length
FOCAL_LENGTH = 668
# cylinder radius, it's much better to use focal length
CYLINDER_RADIUS = FOCAL_LENGTH
RANSAC_THRESHHOLD = 10


In [568]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [569]:
def load_images_from_folder(folder, devide=1):
    paths = sorted(glob.glob(os.path.join(folder,'*')))
    images = [cv2.imread(i) for i in paths]
    images = [cv2.resize(i, (i.shape[1]//devide, i.shape[0]//devide), interpolation=cv2.INTER_AREA) for i in images]
    return images

In [570]:
# # def nonmax_suppression(image):
    
# #     NONMAX_SUPPRESSION_KERNEL = 7
# #     STRIDE=7

# #     result = np.zeros((image.shape))
    
# #     _image = np.lib.stride_tricks.sliding_window_view(image, NONMAX_SUPPRESSION_KERNEL, 1)[:,::STRIDE]
# #     _image = np.lib.stride_tricks.sliding_window_view(_image, NONMAX_SUPPRESSION_KERNEL, 0)[::STRIDE,:]
# #     # print(_image.shape)
# #     _image = np.reshape(_image,(*_image.shape[:2],-1))
# #     _image = np.argmax(_image, axis=2)

# #     mg = np.mgrid[0:_image.shape[0],0:_image.shape[1]]

# #     # y offset: mg[0], x offset: mg[1]
# #     index_y = _image // NONMAX_SUPPRESSION_KERNEL + mg[0] * STRIDE 
# #     index_x = _image % NONMAX_SUPPRESSION_KERNEL + mg[1] * STRIDE 
# #     index_x, index_y = index_x.flatten(), index_y.flatten()


# #     for i, j in zip(index_y, index_x):
# #         result[i][j]=image[i][j]  
        
# #     return  result

# def nonmax_suppression(image, point=300):
#     NONMAX_SUPPRESSION_KERNEL = 15
#     image[:NONMAX_SUPPRESSION_KERNEL//2 , :] = 0
#     image[: ,:NONMAX_SUPPRESSION_KERNEL//2 ] = 0
#     image[-NONMAX_SUPPRESSION_KERNEL//2:, :] = 0
#     image[:,-NONMAX_SUPPRESSION_KERNEL//2: ] = 0

#     result = np.zeros((image.shape))

#     for i in range(point):
#         if(image.max()==0):
#             break
#         # 找到最大的index
#         max_y, max_x = np.unravel_index(image.argmax(), image.shape)

#         # 設點
#         result[max_y][max_x] = image[max_y][max_x]

#         # 上下左右變0
#         image[max_y-NONMAX_SUPPRESSION_KERNEL//2:max_y+NONMAX_SUPPRESSION_KERNEL//2+1,max_x-NONMAX_SUPPRESSION_KERNEL//2:max_x+NONMAX_SUPPRESSION_KERNEL//2+1]=0

    
#     return  result
def nonmax_suppression(image, point=300):
    NONMAX_SUPPRESSION_KERNEL = 21
    reg_img = np.copy(image)
    result = np.zeros((image.shape))

    for i in range(point):
        if(reg_img.max()==0):
            break
        # 找到最大的index
        max_y, max_x = np.unravel_index(reg_img.argmax(), reg_img.shape)

        # 設點
        result[max_y][max_x] = reg_img[max_y][max_x]

        # 上下左右變0
        y_from = max_y-NONMAX_SUPPRESSION_KERNEL//2 if(max_y-NONMAX_SUPPRESSION_KERNEL//2 >= 0 ) else 0
        y_to = max_y+NONMAX_SUPPRESSION_KERNEL//2+1 if(max_y+NONMAX_SUPPRESSION_KERNEL//2+1 <= result.shape[0]) else result.shape[0]
        x_from = max_x-NONMAX_SUPPRESSION_KERNEL//2 if(max_x-NONMAX_SUPPRESSION_KERNEL//2 >= 0 ) else 0
        x_to = max_x+NONMAX_SUPPRESSION_KERNEL//2+1 if(max_x+NONMAX_SUPPRESSION_KERNEL//2+1 <= result.shape[1]) else result.shape[1]

        reg_img[y_from:y_to,x_from:x_to]=0

    
    return  result

In [571]:
def detect_Harris_corner(image, THRESHOLD=10):
    # # parameters
    K = 0.04

    # #  Compute x and y derivatives of image.
    # # rgb to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # R = compute_harris_responce(image)
    
    # gaussion 
    # gradient
    
    I_y = np.zeros(image.shape)
    I_x = np.zeros(image.shape)

    filters.gaussian_filter(image, (1.5, 1.5), (1,0), I_y)
    filters.gaussian_filter(image, (1.5, 1.5), (0,1), I_x)


    # Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # # Compute the sums of the products of derivates at each pixel.
    # # weight : use gaussian
    # # Define the matrix M.
    # # | S_xx  S_xy |
    # # | S_xy  S_yy |
    
    S_xx = filters.gaussian_filter(I_xx, 1.5)
    S_xy = filters.gaussian_filter(I_xy, 1.5)
    S_yy = filters.gaussian_filter(I_yy, 1.5)

    # # Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    # R = detM - K * (traceM * traceM)
    
    R = detM / (traceM +1e-8)

    # # Nonmax Suppression
    R = nonmax_suppression(R)
    
    # # Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255

    # 濾邊界
    R[:20 , :] = 0
    R[: ,:20 ] = 0
    R[-20:, :] = 0
    R[:,-20: ] = 0
    return R

In [572]:
def feature_descriptor(feature_mask, image):

    Gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gau_img = cv2.GaussianBlur(Gray_img, (3,3), 4.5)
    sobel_y = cv2.Sobel(gau_img, cv2.CV_64F, 0, 1)
    sobel_x = cv2.Sobel(gau_img, cv2.CV_64F, 1, 0)
    _, angle = cv2.cartToPolar(sobel_x, sobel_y, angleInDegrees=True)
    # print(angle.dtype)
    DESCRIPTOR_SIZE = 40
    KERNEL = 5

    # descriptor size: 40*40
    feature_point_y, feature_point_x = np.nonzero(feature_mask)
    index_to_get = [KERNEL//2 + i * KERNEL for i in range(DESCRIPTOR_SIZE//KERNEL)]
    # 5*5 sum/mean -> 8*8(descriptor matrix)

    descriptors = []
    positions = []
    for y, x in zip(feature_point_y, feature_point_x):
        y, x = int(y), int(x)
        rot_M = cv2.getRotationMatrix2D((x,y),angle[y,x],1) # (旋轉中心),旋轉角度,縮放比例
        img_rotate = cv2.warpAffine(gau_img,rot_M,(gau_img.shape[1],gau_img.shape[0]), flags = cv2.INTER_NEAREST)

        # 取feature_point-20~+19之間的值，做boxfilter
        big_matrix = img_rotate[y-DESCRIPTOR_SIZE//2:y+DESCRIPTOR_SIZE//2, x-DESCRIPTOR_SIZE//2:x+DESCRIPTOR_SIZE//2]
        big_matrix = cv2.boxFilter(big_matrix, -1, (KERNEL, KERNEL))
        # print(big_matrix.dtype)
        # size = (8, 8)
        descriptor = np.zeros((DESCRIPTOR_SIZE//KERNEL, DESCRIPTOR_SIZE//KERNEL))
        for i in range(DESCRIPTOR_SIZE//KERNEL):
            for j in range(DESCRIPTOR_SIZE//KERNEL):
                descriptor[i][j] = big_matrix[index_to_get[i]][index_to_get[j]]
        descriptor = (descriptor - np.mean(descriptor)) / np.std(descriptor)
        descriptors.append(descriptor)
        positions.append([y, x])

    return np.array(descriptors), np.array(positions)

In [573]:
def find_match_poing(descriptors_1, positions_1, descriptors_2, positions_2, LOW_THERESHOLD=3):
    match_point = []
    # match_dis = []
    
    for ds1, ps1 in zip(descriptors_1, positions_1):
        min_dis = 99999
        min_pos = [-1, -1]

        for ds2, ps2 in zip(descriptors_2, positions_2):
            dis = np.linalg.norm(ds1 - ds2)

            if(dis<min_dis):
                min_dis = dis
                min_pos = np.array([*ps1, *ps2])
                
        if(min_dis<LOW_THERESHOLD):
            match_point.append(min_pos)
            # match_dis.append(min_dis)
    return match_point

In [574]:
def matching(descriptors_1, positions_1, descriptors_2, positions_2):
    # print('find_match_poing1')
    match_point_to_1 = find_match_poing(descriptors_1, positions_1, descriptors_2, positions_2)
    
    # print('find_match_poing2')
    match_point_to_2 = find_match_poing(descriptors_2, positions_2, descriptors_1, positions_1)

    # print('merge')
    match_point = []
    for match_1 in match_point_to_1:
        for match_2 in match_point_to_2:
            dis_1 = np.linalg.norm(match_1[:2]-match_2[-2:])
            dis_2 = np.linalg.norm(match_1[-2:]-match_2[:2])
            
            if(dis_1+dis_2==0):
                match_point.append(match_1)
                
    return np.array(match_point)
    
    # [pos1_y, pos1_x, pos2_y, pos2_x]
    

In [575]:
def draw_point(img, R, output, outputName, r, g, b):
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if (R[i][j]>1):
                img = cv2.circle(img, (j, i), 2, (b, g, r), 2)
    if(output):
        cv2.imwrite(f"{outputName}.jpg", img)

In [576]:
def output_matching_image(images):
    print('Detecting Harris corner.')
    imgs_detect_Harris_corner = [detect_Harris_corner(i) for i in images]
    
    print('Genetating images feature descriptor.')
    # for i, j in zip(images, imgs_detect_Harris_corner):
    #     draw_point(i, j, 0, '', 255, 0, 0) 
    dsps = [feature_descriptor(i, j)for i,j in zip(imgs_detect_Harris_corner, images)]
    match_points = []

    print('Matching descriptor.')
    for i in range(1, len(images)):
        match_points.append(matching(dsps[i-1][0], dsps[i-1][1], dsps[i][0], dsps[i][1]))
    return match_points

In [577]:
def draw_match_line(imgs, match_points):
    for i in range(len(match_points)): 
        img_concat = np.concatenate((imgs[i], imgs[i+1]), axis=1)
        img_w = imgs[i].shape[1]
        for j in match_points[i]:
            img_concat = cv2.line(img_concat, (j[1], j[0]), (j[3]+img_w, j[2]), (255,0,0), 1)

        cv2.imwrite(f'result{i}_{i+1}.jpg', img_concat)

In [578]:
# it's better to use random sample
def nonRANSAC(mp, th = RANSAC_THRESHHOLD): # mp = match pairs set of all images
    best_mp = []
    
    for i in range(len(mp)): # every pairs of different img img_x & img_x+1
        best_vote = 0
        best_mp.append(mp[i][0])
        for j in range(len(mp[i])): # find the best pair of the two images
            vote = 0
            temp_pair = np.array(mp[i][:, 0:2])
            trans_y = mp[i][j][2] - mp[i][j][0]
            trans_x = mp[i][j][3] - mp[i][j][1]
            temp_pair[0] += trans_y 
            temp_pair[1] += trans_x
            for k in range(len(temp_pair)):
                if(k == j):
                    continue
                if(la.norm(temp_pair[k]) < th):
                    vote += 1

            if(vote > best_vote):
                best_vote = vote
                best_mp[i] = mp[i][j]

    return best_mp

In [579]:
def project_to_cylinder(img, f = FOCAL_LENGTH, s = CYLINDER_RADIUS):
    height, width = img.shape[:2]
    cylinder_proj = np.zeros(shape=img.shape, dtype=np.uint8)
    
    for y in range(height):
        py = -y + height//2
        for x in range(width):
            px = x - width//2
            cylinder_proj[-int(s*py/math.sqrt(px**2+f**2) - height//2)][int(s*math.atan(px/f) + width//2)] = img[y][x]
    
    return cylinder_proj

In [580]:
def crop_image_black_edge(img, f = FOCAL_LENGTH, s = CYLINDER_RADIUS):
    h, w = img.shape[:2]
    top = -int(s*(h//2)/math.sqrt((-w//2)**2+f**2) - h//2)
    down = -int(s*(-h//2)/math.sqrt((-w//2)**2+f**2) - h//2)
    left = int(s*math.atan((-w//2)/f) + w//2)
    right = int(s*math.atan((w//2)/f) + w//2)
    crop_img = img[top:down, left:right] # crop 4 sides
    # crop_img = img[:, left:right] # only crop 2 sides, left to right

    return crop_img

In [581]:
# it's better to use random sample
def nonRANSAC(mp, th = RANSAC_THRESHHOLD): # mp = match pairs set of all images
    best_mp = []

    for i in range(len(mp)): # every pairs of different img img_x & img_x+1
        best_vote = 0
        best_dis = 99999
        
        best_mp.append(mp[i][0])
        for j in range(len(mp[i])): # find the best pair of the two images
            vote = 0
            total_dis = 0
            temp_pair = np.array(mp[i][:, :2])
            trans_y = mp[i][j][2] - mp[i][j][0]
            trans_x = mp[i][j][3] - mp[i][j][1]
            # print("pair = ({}, {}, {}, {})".format(mp[i][j][0], mp[i][j][1], mp[i][j][2], mp[i][j][3]))
            # print("trans = {}, {}".format(trans_y, trans_x))
            temp_pair[:, 0] = temp_pair[:, 0] + trans_y - mp[i][:, 2]
            temp_pair[:, 1] = temp_pair[:, 1] + trans_x - mp[i][:, 3]
            # print("temp_pair.len = {}".format(len(temp_pair)))
            for k in range(len(temp_pair)):
                # print("k_pair = {}, norm = {}".format(temp_pair[k], la.norm(temp_pair[k])))
                if(la.norm(temp_pair[k]) < th):
                    vote += 1
                    total_dis += la.norm(temp_pair[k])**2

            total_dis = math.sqrt(total_dis)

            if(vote >= best_vote):
                if(total_dis < best_dis and total_dis > 0):
                    best_dis = total_dis
                    best_vote = vote
                    best_mp[i] = mp[i][j]


    # for z in range(len(best_mp)):
    #     print("mp[0]: {}, best[0]: {}".format(mp[z][0], best_mp[z]))

    return best_mp
        

In [582]:
#  img2       img1            img2+img1
#  _____      _____           _____
# |     |    |.    |  __\    |    _|___   shift = (c,d) - (a,b)
# |    .|    |     |     \   |   |.|   |
# |_____|    |_____|  __ /   |___|_|   |
#  (c,d)      (a,b)     /        |_____|
# 
# y shift > 0, means img1 move down
# x shift > 0, means img1 move right
# in our case, our images are counterclockwise and equal size, so x shift must > 0

def stitch_image(img1, img2, shift): # img1 may be bigger, img2 is always the same size
    img1_padding = [
        (shift[0], 0) if shift[0] > 0 else (0, -shift[0]), 
        (shift[1], 0) if shift[1] > 0 else (0, -shift[1]), 
        (0, 0)
    ]
    padded_img1 = np.lib.pad(img1, img1_padding, 'constant', constant_values=0)

    h_2 = padded_img1.shape[0] - img2.shape[0]
    w_2 = padded_img1.shape[1] - img2.shape[1]
    img2_padding = [
        (h_2, 0) if shift[0] < 0 else (0, h_2), 
        (w_2, 0) if shift[1] < 0 else (0, w_2), 
        (0, 0)
    ]
    padded_img2 = np.lib.pad(img2, img2_padding, 'constant', constant_values=0)

    new_img = alpha_blending(padded_img1, padded_img2, img2.shape,shift)

    # padded_img1[:img2.shape[0], :img2.shape[1]] = img2

    return new_img
    # for i in range(len(match_points)): 
    #     img_concat = imgs[i+1]

    #     img_w = imgs[i].shape[1]
    #     for j in match_points[i]:
    #         img_concat = cv2.line(img_concat, (j[1], j[0]), (j[3]+img_w, j[2]), (255,0,0), 1)

    #     cv2.imwrite(f'result{i}_{i+1}.jpg', img_concat)

In [583]:
def alpha_blending(img1, img2, img2_shape, shift):
    # blending area  = (shift[0], shift[1]) ___ (shift[0], img2.shape[1])
    #                                      |   |
    #             (img2.shape[0], shift[1])|___|(img2.shape[0], img2.shape[1])
    #
    #     (img1.shape[0]-img2_size_h, shift[1]) ___ (img1.shape[0]-img2_size_h, img2.shape[1])
    #                                          |   |
    #            (img1.shape[0]-as_h, shift[1])|___|(img1.shape[0]-as_h, img2.shape[1])
    as_h, as_w = abs(shift[0]), abs(shift[1])
    img2_size_h, img2_size_w = img2_shape[:2]
    img_concat = np.zeros(shape=img1.shape, dtype=np.uint8)
    img_concat = np.add(img_concat, img1)
    img_concat = np.add(img_concat, img2)

    w = img2_size_w - as_w

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)

    for i in range(w):
        # img_concat[as_h:img2_size_h, w+i] = (0,0,0)
        if(shift[0] < 0):
            # y0 = img1.shape[0] - img2_size_h
            y0 = as_h
            y1 = img1.shape[0] - as_h
            img_concat[y0:y1, as_w+i] = img2[y0:y1, as_w+i]*(w-i)/w + img1[y0:y1, as_w+i]*i/w
            # img_concat[y0:y1, as_w+i] = 0
        else:
            img_concat[as_h:img2_size_h, as_w+i] = img2[as_h:img2_size_h, as_w+i]*(w-i)/w + img1[as_h:img2_size_h, as_w+i]*i/w 

    img1 = img1.astype(np.uint8)
    img2 = img2.astype(np.uint8)
    img_concat = img_concat.astype(np.uint8)

    cv2.imwrite("test_blending.jpg", img_concat)

    return img_concat

Main

In [584]:
# load images
# images = load_images_from_folder('img/csie')
# images = load_images_from_folder('img/gym/2-2', 10)
images = load_images_from_folder('img/ours/1', 10)

cylinder_images = [crop_image_black_edge(project_to_cylinder(i)) for i in images]

# return match point 
match_points = output_matching_image(cylinder_images)


Detecting Harris corner.
Genetating images feature descriptor.
Matching descriptor.


In [585]:
match_points

[array([[ 29, 165,  41, 354],
        [ 39,  25,  22, 116],
        [ 54,  44,  65, 233],
        [ 54, 180,  65, 368],
        [ 56,  58,  68, 246],
        [ 58, 115,  69, 304],
        [ 58, 131,  69, 321],
        [ 62, 192,  73, 380],
        [ 71, 222,  81, 409],
        [ 73, 189,  84, 377],
        [ 79,  66,  91, 255],
        [ 80,  26, 135, 214],
        [ 85,  42, 140, 230],
        [ 87, 253,  97, 442],
        [ 95, 236, 105, 423],
        [100,  61, 111, 250],
        [107,  38, 119, 227],
        [108,  26, 178, 160],
        [108, 159, 119, 348],
        [108, 252, 118, 440],
        [109, 107, 121, 296],
        [109, 174, 120, 380],
        [111,  82, 122, 271],
        [123,  25,  47, 215],
        [123,  65, 135, 253],
        [128,  41,  52, 232],
        [129, 219, 139, 407],
        [133, 252, 143, 440],
        [141, 236, 151, 424],
        [147,  26, 158, 215],
        [150,  37,  25, 189],
        [152, 105, 162, 294],
        [152, 232, 162, 420],
        [1

In [586]:
# draw_match_line(cylinder_images, match_points)

In [587]:
ransac_mp = nonRANSAC(match_points[1:])
shifts = [ransac_mp[i][2:] - ransac_mp[i][:2] for i in range(len(ransac_mp))]
temp_img = cylinder_images[0]
for t in range(0, len(ransac_mp)):
    temp_img = stitch_image(temp_img, cylinder_images[t+1], shifts[t])
    cv2.imwrite(f"shift_result{t}_{t+1}.jpg", temp_img)
# stitch_image(cylinder_images[0], cylinder_images[1], (-53, 125))

In [588]:
# temp_img = cylinder_images[0].copy()
print(temp_img.shape)
threshold_to_clip = cylinder_images[0].shape[1]
temp_img_sum = np.sum(temp_img, axis=2)

height_from_top = 0
for i in temp_img:
    if(len(np.where(i==0)[0])<threshold_to_clip):
        break
    height_from_top+=1

height_from_bot = 0
for i in temp_img[::-1]:
    if(len(np.where(i==0)[0])<threshold_to_clip):
        break
    height_from_bot+=1

print(height_from_top, height_from_bot)

(389, 2036, 3)
84 84


In [589]:
crop_temp_img = temp_img[height_from_top:temp_img.shape[0]-height_from_bot,:,:]
cv2.imwrite('crop_temp_img.jpg',crop_temp_img)

True